In [6]:
import os, sys
import json

## Initialisation

In [7]:
from azureml.core import Workspace

# ws = Workspace.from_config()

ws = Workspace.get(name="Walmart-Sales", 
    subscription_id =  "ef0073f1-56e3-462f-80b3-3beb320211e4",
    resource_group = "Walmart-1"
)

In [5]:
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print(compute.name, ":", compute.type)

compute-1 : AmlCompute


In [4]:
ws.compute_targets

{'compute-1': AmlCompute(workspace=Workspace.create(name='Walmart-Sales', subscription_id='ef0073f1-56e3-462f-80b3-3beb320211e4', resource_group='Walmart-1'), name=compute-1, id=/subscriptions/ef0073f1-56e3-462f-80b3-3beb320211e4/resourceGroups/Walmart-1/providers/Microsoft.MachineLearningServices/workspaces/Walmart-Sales/computes/compute-1, type=AmlCompute, provisioning_state=Succeeded, location=southeastasia, tags={})}

## Creating Environment

In [13]:
from azureml.core import Environment

In [14]:
path_to_requirements = os.path.abspath("../requirments.txt")

In [15]:
env = Environment.from_pip_requirements("walmart2", path_to_requirements)

In [16]:
env.register(ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210220.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "walmart2",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
   

## Creating Experiments

In [1]:
from azureml.core import Experiment

# create an experiment variable
experiment = Experiment(workspace = ws, name = "my-experiment")

# start the experiment
run = experiment.start_logging()

# experiment code goes here



# Log the row count
run.log('observations', 10)

# end the experiment
run.complete()

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (numpy 1.21.5 (c:\users\xa155xb\anaconda3\envs\walmart2\lib\site-packages), Requirement.parse('numpy<=1.19.3; sys_platform == "win32"'), {'azureml-dataset-runtime'}).


NameError: name 'ws' is not defined

#### Running a Script as an Experiment

In [ ]:
AML_ENV_NAME = "environment-1"
AML_COMPUTE_TARGET = "compute-3"
VM_SIZE = "STANDARD_D2_V2"

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

# Create a script config
script_config = ScriptRunConfig(source_directory="experiment_folder",
                                script='experiment.py')

# Selecting the environment
experiment_env = Environment.get(
    workspace = ws,
    name = AML_ENV_NAME
)

script_config.run_config.environment = experiment_env

# Selecting compute target
try:
    compute_target = ComputeTarget(
        workspace = ws,
        name = AML_COMPUTE_TARGET
    )
except ComputeTargetException:
    # Create the new compute target
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size = VM_SIZE,
        min_nodes = 0,
        max_nodes = 2
    )
    compute_target = ComputeTarget.create(
        workspace = ws,
        name = AML_COMPUTE_TARGET,
        provisioning_configuration= compute_config
    )
    compute_target.wait_for_completion(
        show_output = True,
        timeout_in_minutes=20
    )


script_config.run_config.target = compute_target

# create an experiment variable
experiment = Experiment(workspace = ws, name = "prophet")

# submit the experiment
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)

## Retrieving and Viewing Logged Metrics

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

In [ ]:
files = run.get_file_names()
print(json.dumps(files, indent=2))

In [ ]:
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')